[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scottlevie97/python_FVM_CSM/blob/newBoundaryConditions/seperate_notebooks/_06_Solution_Algorithm.ipynb)

# 6. **Solution Algorithm**
---


We've described the $A$-matrices and $b$-matrices for:

- All points within the mesh
- Both the x and y momentum equations.

How will these matrices be used to solve the momentum equations? This notebook describes the procedure that finds a solution to the equations.  

## **Step 1:** Initialise Matrix

We'll create both the A and b matrices for the x and y equations

### Creating the $A$-matrix

Previously we have created the A matrix for internal cells. This is as follows:


In [15]:
from functions.setup import *
from ipynb.fs.defs._05_Fixed_Traction_BCs import *

# x A matrix
A_x = A("x").createMatrix()

# y A matrix
A_y = A("y").createMatrix()

### Add BC Terms to $A$ and $b$ Matrices

For each cell on boundaries, we've created functions that take in A and b matrices and assign the a-terms and b-terms to these matrices. We need to write a function that loops through each point <code>k</code> in the mesh and assigns the correct term.

In [16]:
from ipynb.fs.defs._04_Fixed_Displacement_BCs import *

In [17]:
def cell_boundary_selection_A(A_matrix, k, boundaries, xy, U_old, U_old_old, U_previous):

    if BC_settings(boundaries[0]).traction:
        A_matrix = traction_cell_BCs_A(A_matrix, k, boundaries, xy)

    elif BC_settings(boundaries[0]).fixed_displacement:
        A_matrix = displacement_cell_BCs_A(A_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

    return A_matrix

def cell_boundary_selection_b(
    b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous
):

    if BC_settings(boundaries[0]).traction:
        b_matrix = traction_cell_BCs_b(
            b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous
        )

    elif BC_settings(boundaries[0]).fixed_displacement:
        b_matrix = displacement_cell_BCs_b(
            b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous
        )

    return b_matrix

The following functions loop through each cell <code>k</code> to determine if the cell is an internal, boundary or corner cell. Then the BC terms are applied. These functions are long but straightforward. 


In [18]:
def boundary_conditions_A(A_matrix, U_previous, U_old, U_old_old, xy):
    for k in np.arange(0,(nx)*(ny)):  

        #Bottom left corner coefficients        
        if cell_index().bottom_left_corner(k):  

            boundaries = ["b", "l"]
            A_matrix = cell_corner_BCs_A(A_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)          

        #Bottom right corner coefficients            
        elif  cell_index().bottom_right_corner(k):
            
            boundaries = ["b", "r"]
            A_matrix = cell_corner_BCs_A(A_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)          

        #Top left corner coefficients            
        elif  cell_index().top_left_corner(k):
            
            boundaries = ["t", "l"]
            A_matrix = cell_corner_BCs_A(A_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)          
            
        #Top right corner coefficients            
        elif  cell_index().top_right_corner(k):

            boundaries = ["t", "r"]
            A_matrix = cell_corner_BCs_A(A_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)          
            
        # Center Bottom Boundaries
        elif  cell_index().center_bottom(k):
            
            boundaries = ["b"]
            A_matrix = cell_boundary_selection_A(A_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        # Center Top Boundaries
        elif cell_index().center_top(k):
            boundaries = ["t"]

            A_matrix = cell_boundary_selection_A(A_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        # Center Left Boundaries
        elif  cell_index().center_left(k):
            boundaries = ["l"]

            A_matrix = cell_boundary_selection_A(A_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        # Center Right Boundaries
        elif  cell_index().center_right(k):
            boundaries = ["r"]

            A_matrix = cell_boundary_selection_A(A_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)


    return A_matrix


In [19]:
def boundary_conditions_b(b_matrix, U_previous, U_old, U_old_old, xy):
    for k in np.arange(0,(nx)*(ny)):   # j is the cell number

    #    #Bottom left corner coefficients        
        if cell_index().bottom_left_corner(k):  

            boundaries = ["b", "l"]
            
            b_matrix = cell_corner_BCs_b(b_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)          

        #Bottom right corner coefficients            
        elif  cell_index().bottom_right_corner(k):
            boundaries = ["b", "r"]
            
            b_matrix = cell_corner_BCs_b(b_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)

        #Top left corner coefficients            
        elif  cell_index().top_left_corner(k):
            boundaries = ["t", "l"]
        
            b_matrix = cell_corner_BCs_b(b_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)

        #Top right corner coefficients            
        elif  cell_index().top_right_corner(k):  
            boundaries = ["t", "r"]
            
            b_matrix = cell_corner_BCs_b(b_matrix, k, boundaries, xy, U_previous, U_old, U_old_old)

        # Center Bottom Boundaries
        elif  cell_index().center_bottom(k):
            boundaries = ["b"]

            b_matrix = cell_boundary_selection_b(b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        # Center Top Boundaries
        elif cell_index().center_top(k):
            boundaries = ["t"]

            b_matrix = cell_boundary_selection_b(b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        # Center Left Boundaries
        elif  cell_index().center_left(k):
            boundaries = ["l"]

            b_matrix = cell_boundary_selection_b(b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        # Center Right Boundaries
        elif  cell_index().center_right(k):
            boundaries = ["r"]

            b_matrix = cell_boundary_selection_b(b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous)

        else:

            b_matrix[k] = (
                            A.b_temporal(U_old, U_old_old, k, xy)
                            +
                            A.b_force(k, xy, U_previous)
                        )

    return b_matrix


Using this function to assign BC values to A and b matrices: 

In [20]:
# Initialise $b$-matrix 
b_x = np.zeros([ny*nx,1])
b_y = np.zeros([ny*nx,1])

# Assign boundary conditions to matrices
A_x = boundary_conditions_A(A_x, U_previous, U_old, U_old_old, "x")
b_x = boundary_conditions_b(b_x, U_previous, U_old, U_old_old, "x")

A_y = boundary_conditions_A(A_y, U_previous, U_old, U_old_old, "y")
b_y = boundary_conditions_b(b_y, U_previous, U_old, U_old_old, "y")

Now that Both A matrices and b matrices are fully formed, how will we solve for the displacement field?

## **Step 2:** Solve Matrix

In order solve for a solution displacement field $(u , v)$. We know:

$$
\begin{bmatrix} a \end{bmatrix}_x
\begin{bmatrix} u \end{bmatrix}
 = \begin{bmatrix} b \end{bmatrix}_x
$$

$$
\begin{bmatrix} a \end{bmatrix}_y
\begin{bmatrix} v \end{bmatrix}
 = \begin{bmatrix} b \end{bmatrix}_y
$$



To solve these matrices we'll use a function from <code>numpy</code>:  <code>linalg.solve</code>:


In [21]:
from numpy.linalg import solve

# solve x displacement
u = solve(A_x, b_x)
# convert to numpy array
u = np.array(u)

# solve y displacement
v = solve(A_y, b_y)
# convert to numpy array
v = np.array(v)

# Update U_new with new x and y displacements
U_new = np.hstack((u, v))

However, we also know that:

- $ \begin{bmatrix} b \end{bmatrix} _x $ is a function of $v$

- $ \begin{bmatrix} b \end{bmatrix}_y $ is a function of $u$

Perhaps you can see a problem: when you solve for $u$ and $v$ using the matrices, these fields may differ from those used in the b-terms.

*So how do we solve this?*

One solution is to pass the solution field back into the b-terms, create new b-matrices, and resolve the displacement fields.

But again, the solution field may still be different to the field used to create the b matrix.

$$
\begin{align*}
\begin{bmatrix} a \end{bmatrix}_x
\begin{bmatrix} u_0 \end{bmatrix}
 = \begin{bmatrix} b(v_0) \end{bmatrix}_x  \rightarrow &   \space \space \space \space u_1  &\\
\begin{bmatrix} a \end{bmatrix}_y
\begin{bmatrix} v_0 \end{bmatrix}
 = \begin{bmatrix} b(u_0) \end{bmatrix}_y  \rightarrow &  \space \space \space \space v_1 &\\
 \space \space & \space \space \space \downarrow &\\
\begin{bmatrix} a \end{bmatrix}_x
\begin{bmatrix} u_1 \end{bmatrix}
= & \begin{bmatrix} b(v_1) \end{bmatrix}_x \rightarrow &  u_2  \\
\begin{bmatrix} a \end{bmatrix}_y
\begin{bmatrix} v_1 \end{bmatrix}
= & \begin{bmatrix} b(u_1) \end{bmatrix}_y \rightarrow &  v_2 \\
\end{align*}
$$


If we do this repeatedly $i$ times (subscript), the difference between the solution displacement ($i+1$) and the displacement used to generate the b matrix  ($i$) should decrease. Each repetition of this process is called an iteration.

We need to discuss how to measure this difference and set a tolerance for how small this difference can be before we consider the solution to be solved.

## **Step 3**: Calculate Residual

The difference between the displacement fields in adjacent iterations is known as the residual.

There are multiple ways how to calculate the residual. A standard method is to find the Root Mean Squared Error (RMSE) and "normalise" this against the largest value. Effectively, we can say the mean of the squared errors of all points *$x$* times the size of the largest displacement, where the *$x$* value is the tolerance we want. 

Firstly we square the differences so that all values are positive:

In [22]:
# This give a 2D array of the squared difference

sqrDiff = (U_new - U_previous)**2

Then we find the mean of this array

In [23]:
np.mean((U_new - U_previous)**2)

1.1264906249953836e-10

Now we need to return this value into displacement units by finding the square root

In [24]:
import math 
math.sqrt(np.mean((U_new - U_previous)**2))

1.0613626265303408e-05

This value is the RMSE; however, depending on the geometry of the test case used, this value can mean different things. So, we need to normalise this value by dividing it by the largest displacement (<code>normFactor</code>). A variable <code>SMALL</code> is added to <code>normFactor</code> to avoid dividing by zero.

In [25]:
SMALL = 1e-16

normFactor = np.max(U_new) + SMALL

residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor

To Summarise: 

1. We calculate the difference between each point on the mesh in the displacement solution field for iteration $i$ and $i+1$.
2. We square these values so that positive and negative values are treated equally.
3. We take an average of these values (*** why do we do this?)
4. We take the square root of this to bring the residual back into the unit used in the displacement field.
5. We normalise this by the largest displacement in the solution field. 

So our residual is a measure of the percentage of the root mean squared error of displacement fields, for two adjacent iterations, in comparison to the largest displacement in the field. If we enforced that the residual must be 0.0000001, i.e, 0.00001 %, we can say that the difference between the solution fields are reasonably small.

The value we enforce the residual to drop to before accepting that reasonable solution fields are found is called the **tolerance**. When this is reached, the solution is said to be **converged**.

## **Step 4**: Combine into Momentum Loop

To enforce the residual to drop to a certain tolerance. We need to create a loop that performs these iterations to solve the momentum equations, the **momentum loop**.


Let's think of how this loop will be solved:

        create A matrices
        
        while (!converged):

                # set previous displacement solutions
                U_previous = U_new 

                # x equation
                create b matrix using previous v
                solve for u

                # y equation
                create b matrix using previous u
                solve for v

                Calculate the residual

                if (residual < tolerance>):
                        converged = True
                else
                        converged = False



In [26]:
# Initialise fields
b_x = np.zeros([(ny)*(nx),1])
b_y = np.zeros([(ny)*(nx),1])

U_new = initalise_U_field(nx, ny)
U_old = initalise_U_field(nx, ny)
U_old_old = initalise_U_field(nx, ny)

# Set tolerance
tolerance = 1e-6

# Iteration counter 
icorr = 0

# Maximum iteration limit (This will be increased for final implementation)
maxcorr = 5

# Create A matrices:
A_x = A("x").createMatrix()   
A_y = A("y").createMatrix()   

# Add boundary conditions to A matrices
A_x = boundary_conditions_A(A_x, U_previous, U_old, U_old_old, "x")
A_y = boundary_conditions_A(A_y, U_previous, U_old, U_old_old, "y")

while True:

    # set previous displacement solutions
    U_previous = U_new

    # x-equation
    # Create b matrices
    b_x = boundary_conditions_b(b_x, U_previous, U_old, U_old_old, "x")

    # solve for u
    u = solve(A_x, b_x)
    u = np.array(u)

    # y-equation
    # Create b matrices        
    b_y = boundary_conditions_b(b_y, U_previous, U_old, U_old_old, "y")

    # solve for v
    v = solve(A_y, b_y)
    v = np.array(v)
        
    # Update U_new with new x and y displacements
    U_new = np.hstack((u, v))

    ## Calculate the residual of each iteration    
    normFactor = np.max(U_new)

    residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor

    # print values
    print("Iteration: {:01d},\t Residual = {:.20f},\t normFactor = {:.20f}".format(icorr, residual, normFactor))

    # Convergence check
    if residual < tolerance:

        print("Solution has converged")

        break
        
    elif icorr > maxcorr:
        
        break            
    
    icorr = icorr + 1


/tmp/ipykernel_12136/2379744722.py:53: RuntimeWarning: divide by zero encountered in double_scalars
  residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor


Iteration: 0,	 Residual = inf,	 normFactor = 0.00000000000000000000
Iteration: 1,	 Residual = 0.51523940885836416737,	 normFactor = 0.00000048749999999975
Iteration: 2,	 Residual = 20.87325183127153138685,	 normFactor = 0.00000048749999999975
Iteration: 3,	 Residual = 0.25476369375290025543,	 normFactor = 0.00000097499999999926
Iteration: 4,	 Residual = 10.17080655277777090362,	 normFactor = 0.00000097499999999926
Iteration: 5,	 Residual = 0.16833692481888815062,	 normFactor = 0.00000146249999999799
Iteration: 6,	 Residual = 6.64817614566296466450,	 normFactor = 0.00000146249999999799


The residual tends to change substantially from iteration to iteration, so it's useful to calculate the moving average of the residual. This will be added to the momentum loop in the next notebook.

Once the momentum loop has converged, the solution algorithm moves to the next time-step. This procedure is known as the **time loop**, this will be discussed in the next notebook.

Below is an overview of the solution algorithm used in this solver:

<code>
    
Solution Algorithm 
---------
1. Switch to next time-step
2. Switch to next iteration
3. Initialise A & b matrices with internal cell values
4. Apply boundary conditions to A & b matrices
5. Solve for the displacement field
6. Calculate residual
7. **If** converged **then**:
8.  Go to next time-step (line 1)
9. **else** 
10.  Go to next iteration (line 2)

</code>

